# KAKENの研究機関マスタをローカルのMariaDBに保存するプログラム

### 事前準備
- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、configparserで ./config.ini として保存しておく。

### ここから本編
bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [3]:
# encoding: utf-8
from lxml import etree
import pandas as pd

In [5]:
tree = etree.parse('grants_masterxml_kaken/institution_master_kakenhi.xml')

institutionlist = []
for institution_table in tree.iterfind("institution_table"):
    for institution in institution_table.iterfind("institution"):
        institution_name = institution.find("name[@lang='ja']").text
        institution_niicode = institution.find("code[@type='nii']").text

        row = [
            institution_niicode,
            institution_name,
        ]
        
        institutionlist.append(row)
            
df = pd.DataFrame(institutionlist)
df.columns = ['institution_niicode', 'institution_name']
df

,institution_niicode,institution_name
0,0010101,北海道大学
1,0010102,北海道教育大学
2,0010103,室蘭工業大学
3,0010104,小樽商科大学
4,0010105,帯広畜産大学
5,0010106,北見工業大学
6,0010107,旭川医科大学
7,0011101,弘前大学
8,0011201,岩手大学
9,0011301,東北大学


重複のデータがあるかどうか

In [6]:
df.duplicated().any()

True

重複データがあった。dropしておく。

In [7]:
df = df.drop_duplicates()
df.duplicated().any()

False

In [8]:
df.institution_niicode.value_counts()

0082679    1
0042651    1
0082699    1
0082201    1
0037602    1
0034318    1
0031101    1
0027401    1
0094601    1
0043941    1
0030110    1
0054601    1
0056401    1
382897     1
0033943    1
0037408    1
0051201    1
0074325    1
0042677    1
0092720    1
0037117    1
0032105    1
0035313    1
0032635    1
0032812    1
0083803    1
0024303    1
0035409    1
0021601    1
0044435    1
          ..
0046104    1
0183102    1
0382657    1
0023103    1
0086401    1
382724     1
0023701    1
0032629    1
0174302    1
0014701    1
0037120    1
0034528    1
0034409    1
0047202    1
0084422    1
0046304    1
0043945    1
0134520    1
0042727    1
0235505    1
0042514    1
0037107    1
0033936    1
0030108    1
0042721    1
0042694    1
0035415    1
382766     1
0092105    1
0171301    1
Name: institution_niicode, Length: 2618, dtype: int64

In [13]:
df = df.astype({
    'institution_niicode': int,
})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2618 entries, 0 to 2617
Data columns (total 2 columns):
institution_niicode    2618 non-null int32
institution_name       2618 non-null object
dtypes: int32(1), object(1)
memory usage: 30.8+ KB


niicodeがユニークなので、インデックスに設定する

In [14]:
df = df.set_index('institution_niicode')
df

,institution_name
institution_niicode,
10101,北海道大学
10102,北海道教育大学
10103,室蘭工業大学
10104,小樽商科大学
10105,帯広畜産大学
10106,北見工業大学
10107,旭川医科大学
11101,弘前大学
11201,岩手大学


ローカルのmariaDBに関する設定ファイルを読み込み（config.iniはgitに上げていないが、事前準備で作られているはず）

In [16]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

データベースにテーブル構造とデータの中身を書き込む

In [17]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('kaken_master_institution', engine, if_exists='replace',
          dtype={
              'institution_niicode': Integer,
              'institution_name': String(256),
                })

2018-05-09 11:52:23,385 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-05-09 11:52:23,386 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 11:52:23,387 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-05-09 11:52:23,387 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 11:52:23,388 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-05-09 11:52:23,389 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 11:52:23,390 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-05-09 11:52:23,390 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 11:52:23,391 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-05-09 11:52:23,392 INFO sqlalchemy.engine.base.Engine {}
2018-05-09 11:52:23,392 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-05-09 11

### おしまい
データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。